In [1]:
import pandas as pd
import numpy as np
import nltk
import re, string
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,
                              GradientBoostingClassifier,
                              HistGradientBoostingClassifier)
from gensim.models import Word2Vec

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn_crfsuite import CRF, scorers, metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mnikh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mnikh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mnikh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df_crime_train= pd.read_csv('/Users/mnikh/Desktop/IUPUI/Study/Data Science/Kaggle final competetion/crime_train.csv')
df_crime_test=pd.read_csv('/Users/mnikh/Desktop/IUPUI/Study/Data Science/Kaggle final competetion/crime_test.csv')

In [3]:
df_crime_train

,id,BEGDATE,NARRATIVE,X,Y,CRIMETYPE
0,170243,10/31/12 0:00,UNK SUSP ENT VICT'S VEH THROUGH UNLOCKED DOOR ...,6470007.500,1763648.500,BTFV
1,176597,12/23/12 0:00,UNK SUSP OPENED SIDE WINDOW TO GAIN ENTRY SUS...,6480901.500,1802058.800,BURG
2,40020,11/9/09 0:00,NaN,6448486.498,1909028.127,BTFV
3,63463,5/8/10 0:00,SUSP ADMITTED THAT HE HAS BEEN PLANNING TO GET...,6375807.998,1896102.879,BURG
4,194886,5/31/13 0:00,UNK S POSSIBLY ENTERED V APARTMENT NO PROPERTY...,6431452.200,1899871.800,BURG
...,...,...,...,...,...,...
99995,29483,8/19/09 0:00,NaN,6461025.400,1863011.000,BTFV
99996,132624,12/13/11 0:00,UNK S REMOVED TAB FROM VEH,6391516.600,1913890.300,BTFV
99997,107653,5/14/11 0:00,UNK SUSP ENTER CAMPUS FROM AN UNK DIRECTION AN...,6428035.500,1921800.300,BURG
99998,68561,6/19/10 0:00,UNK S REMOVED VICTS REGS TAB FROM REAR LIC PLA...,6415960.200,1893347.300,BTFV


In [3]:
from sklearn.model_selection import train_test_split

In [5]:
df_crime_test

,id,BEGDATE,NARRATIVE,X,Y
0,185004,3/5/13 0:00,UNK SUSP SMASHED VIC FRONT DRIVER DOOR TO GAIN...,6435950.100,1815357.400
1,137809,1/23/12 0:00,UNK SUSPS USED AN UNK OBJECT TO PRY AND ATTEMP...,6433527.900,1921769.400
2,159499,7/30/12 0:00,NaN,6473065.700,1820081.500
3,19314,5/30/09 0:00,UNK SUSP SMASHED VICTS VEH WINDOW ENTERED VEH ...,6455558.500,1842435.700
4,6934,2/21/09 0:00,UNK SUSP CUT VIC DOOR SCREW WITH UNK SHARP OBJ...,6439876.300,1808801.400
...,...,...,...,...,...
99995,107216,5/10/11 0:00,UNK SUSP ENTD VICT PROP THROUGH AN UNLOCKED GA...,6376770.600,1919210.000
99996,168476,10/17/12 0:00,UNK SUSP ENTERED LOC VIA POSS UNLOCKED BACK DO...,6450038.500,1829696.500
99997,71026,7/10/10 0:00,UNK SMASHED VICTS CAR WIND AND ATTMPT TO TAKE ...,6398597.200,1921972.200
99998,80246,9/26/10 0:00,UNK SUS BROKE SIDE WINDOW OF V1 VEH OPEN BONN...,6422412.800,1888048.100


In [4]:
df_crime_train['type'] = "train"
df_crime_test['type'] = "test"
df_crime_test['CRIMETYPE'] = "null"

In [5]:
df_crime_data = pd.concat([df_crime_train, df_crime_test])

In [21]:
df_crime_data

,id,BEGDATE,NARRATIVE,X,Y,CRIMETYPE,type
0,170243,10/31/12 0:00,UNK SUSP ENT VICT'S VEH THROUGH UNLOCKED DOOR ...,6470007.500,1763648.500,BTFV,train
1,176597,12/23/12 0:00,UNK SUSP OPENED SIDE WINDOW TO GAIN ENTRY SUS...,6480901.500,1802058.800,BURG,train
2,40020,11/9/09 0:00,NaN,6448486.498,1909028.127,BTFV,train
3,63463,5/8/10 0:00,SUSP ADMITTED THAT HE HAS BEEN PLANNING TO GET...,6375807.998,1896102.879,BURG,train
4,194886,5/31/13 0:00,UNK S POSSIBLY ENTERED V APARTMENT NO PROPERTY...,6431452.200,1899871.800,BURG,train
...,...,...,...,...,...,...,...
99995,107216,5/10/11 0:00,UNK SUSP ENTD VICT PROP THROUGH AN UNLOCKED GA...,6376770.600,1919210.000,null,test
99996,168476,10/17/12 0:00,UNK SUSP ENTERED LOC VIA POSS UNLOCKED BACK DO...,6450038.500,1829696.500,null,test
99997,71026,7/10/10 0:00,UNK SMASHED VICTS CAR WIND AND ATTMPT TO TAKE ...,6398597.200,1921972.200,null,test
99998,80246,9/26/10 0:00,UNK SUS BROKE SIDE WINDOW OF V1 VEH OPEN BONN...,6422412.800,1888048.100,null,test


In [6]:
df_crime_train = df_crime_train.dropna(subset=['NARRATIVE'])

In [7]:
df_crime_train['word_count'] = df_crime_train['NARRATIVE'].apply(lambda x: len(str(x).split()))

In [19]:
def initialization(sentences_data):
    
    sentences_data = sentences_data.lower()
    sentences_data = sentences_data.strip()
    sentences_data = re.compile('<.*?>').sub('', sentences_data)
    sentences_data = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', sentences_data)
    sentences_data = re.sub('\s+', ' ', sentences_data)
    sentences_data = re.sub(r'\[[0-9]*\]', ' ', sentences_data)
    sentences_data = re.sub(r'[^\w\s]', '', str(sentences_data).lower().strip())
    sentences_data = re.sub(r'\d', ' ', sentences_data)
    sentences_data = re.sub(r'\s+', ' ', sentences_data)
    return sentences_data


In [20]:
def Endings(string):
    a = [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

M = WordNetLemmatizer()


In [21]:
def pos_net(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [22]:
def middle_process(string):
    mag = nltk.pos_tag(word_tokenize(string))
    a = [M.lemmatize(tag[0], pos_net(tag[1])) for idx, tag in
         enumerate(mag)]
    return " ".join(a)

In [23]:
def pro_last(string):
    return middle_process(Endings(initialization(string)))

In [ ]:
df_crime_train['clean_text'] = df_crime_train['NARRATIVE'].apply(lambda x: pro_last(x))

df_crime_test = df_crime_train[df_crime_train['type'] == 'test']
df_crime_train = df_crime_train[data_train['type'] == 'train']

In [ ]:
A_train = df_crime_train['NARRATIVE'].squeeze()
A_test = df_crime_test['NARRATIVE'].squeeze()
B_train = df_crime_train['CRIMETYPE'].squeeze()
B_test = df_crime_test['CRIMETYPE'].squeeze()


In [ ]:
A_traintoken= [nltk.word_tokenize(i) for i in A_train]
A_testtoken= [nltk.word_tokenize(i) for i in A_test]


In [ ]:
# Generating Tf-Idf vectors
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(A_train)
X_test_vectors_tfidf = tfidf_vectorizer.transform(A_test)


In [ ]:
#CatBoost Model
cb = CatBoostClassifier(n_estimators=2000,
                        colsample_bylevel=0.06,
                        max_leaves=31,
                        subsample=0.67,
                        verbose=0,
                        thread_count=6,
                        random_state=1234)
cb.fit(X_train_vectors_tfidf, B_train)

y_predict = cb .predict(X_test_vectors_tfidf)
# predict_v = np.where(y_predict == 1, 'BURG', 'BTFV')
df_crime_test['id'] = pd.to_numeric(df_crime_test['id'])
result = pd.DataFrame({'id': df_crime_test.id, 'CRIMETYPE': y_predict})
result.to_csv("/Users/mnikh/Desktop/IUPUI/Study/Data Science/Kaggle final competetion/CatBoostClassifier.csv", index=False)
#pd.DataFrame(y_predict).to_csv("/Users/mnikh/Desktop/IUPUI/Study/Data Science/Kaggle final competetion/CatBoostClassifier.csv")
print("finished CatBoost Model execution")


In [ ]:
#LGBM Model

lgbm = LGBMClassifier(n_estimators=2000,
                      feature_fraction=0.06,
                      bagging_fraction=0.67,
                      bagging_freq=1,
                      verbose=0,
                      n_jobs=6,
                      random_state=1234)
lgbm.fit(X_train_vectors_tfidf, B_train)
y_predict = lgbm .predict(X_test_vectors_tfidf)
#predict_v = np.where(y_predict == 1, 'BURG', 'BTFV')
df_crime_test['id'] = pd.to_numeric(df_crime_test['id'])
result = pd.DataFrame({'id': df_crime_test.id, 'CRIMETYPE': y_predict})
result.to_csv("/Users/mnikh/Desktop/IUPUI/Study/Data Science/Kaggle final competetion/LGBMClassifier.csv", index=False)
#pd.DataFrame(y_predict).to_csv("/Users/mnikh/Desktop/IUPUI/Study/Data Science/Kaggle final competetion/LGBMClassifier.csv")
print("finished LGBM Model execution")

In [ ]:
#RandomForest Model

rf = RandomForestClassifier(n_estimators=500,
                            max_features=0.06,
                            n_jobs=6,
                            random_state=1234)
rf.fit(X_train_vectors_tfidf, B_train)
y_predict = rf.predict(X_test_vectors_tfidf)
#predict_v = np.where(y_predict == 1, 'BURG', 'BTFV')
df_crime_test['id'] = pd.to_numeric(df_crime_test['id'])
result = pd.DataFrame({'id': df_crime_test.id, 'CRIMETYPE': y_predict})
result.to_csv("/Users/mnikh/Desktop/IUPUI/Study/Data Science/Kaggle final competetion/rfclassifier.csv", index=False)
print("finished RF Model execution")
base_estim = DecisionTreeClassifier(max_depth=1, max_features=0.06)

In [ ]:
#AdaBoost Model
ab = AdaBoostClassifier(base_estimator=base_estim,
                        n_estimators=500,
                        learning_rate=0.5,
                        random_state=1234)

ab.fit(X_train_vectors_tfidf, B_train)
y_predict = ab.predict(X_test_vectors_tfidf)
#predict_v = np.where(y_predict == 1, 'BURG', 'BTFV')
df_crime_test['id'] = pd.to_numeric(df_crime_test['id'])
result = pd.DataFrame({'id': df_crime_test.id, 'CRIMETYPE': y_predict})
result.to_csv("/Users/mnikh/Desktop/IUPUI/Study/Data Science/Kaggle final competetion/ab.csv", index=False)
print("finished Ada Boost Model execution")

In [ ]:
#XGBoost Model
xgb = XGBClassifier(n_estimators=2000,
                    tree_method='hist',
                    subsample=0.67,
                    colsample_level=0.06,
                    verbose=0,
                    n_jobs=6,
                    random_state=1234)

xgb.fit(X_train_vectors_tfidf, B_train)
y_predict = xgb.predict(X_test_vectors_tfidf)
#predict_v = np.where(y_predict == 1, 'BURG', 'BTFV')
df_crime_test['id'] = pd.to_numeric(df_crime_test['id'])
result = pd.DataFrame({'id': df_crime_test.id, 'CRIMETYPE': y_predict})
result.to_csv("/Users/mnikh/Desktop/IUPUI/Study/Data Science/Kaggle final competetion/xgb.csv", index=False)
print("finished XG Boost Model execution")

In [ ]:
#GradientBoost Model

gbm = GradientBoostingClassifier(n_estimators=2000,
                                 subsample=0.67,
                                 max_features=0.06,
                                 validation_fraction=0.1,
                                 n_iter_no_change=15,
                                 verbose=0,
                                 random_state=1234)

gbm.fit(X_train_vectors_tfidf, B_train)
y_predict = gbm.predict(X_test_vectors_tfidf)
#predict_v = np.where(y_predict == 1, 'BURG', 'BTFV')
df_crime_test['id'] = pd.to_numeric(df_crime_test['id'])
result = pd.DataFrame({'id': df_crime_test.id, 'CRIMETYPE': y_predict})
result.to_csv("/Users/mnikh/Desktop/IUPUI/Study/Data Science/Kaggle final competetion/gbm.csv", index=False)
print("finished GradientBoost Model execution")
